In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT_DIR = '/content/drive/My Drive/YOLO_Training'

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import os
import numpy as np
import torch
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Set device to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
!nvidia-smi

Mon Mar 17 01:34:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
subset_config_path = os.path.join(ROOT_DIR, "config.yaml")
!ls "/content/drive/MyDrive/YOLO_Training/"

 capstone_project	      Figuring_Out_Y_Axis.ipynb   T5_Training_Dataset   YOLO_Training.ipynb
 config.yaml		      predictions		  Test_Images
'Figuring Out Plot.ipynb'     runs_old			  yolo11m.pt
'Figuring_Out X_Axis.ipynb'  'T5 Model Training.ipynb'	  YOLO_to_OCR.ipynb


In [ ]:
# Best hyperparameters from HPO
best_lr = 0.0005
best_momentum = 0.9
best_wd = 0.0005

In [ ]:
# Load the YOLOv11m model; ensure 'yolo11m.pt' is in your ROOT_DIR
model = YOLO(os.path.join(ROOT_DIR, "yolo11m.pt"))
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [ ]:
# Full training on the complete dataset config (config.yaml)
results = model.train(
    data = os.path.join(ROOT_DIR, "config.yaml"), # Full dataset config file
    epochs = 100,# Full training epochs
    imgsz = 800, # Image resolution
    batch = 8, # Adjust based on your GPU memory
    lr0 = best_lr, # Best learning rate from HPO
    momentum = best_momentum, # Best momentum from HPO
    weight_decay = best_wd, # Best weight decay from HPO
    optimizer = "SGD", # Using SGD optimizer
    workers = 2, # Number of dataloader workers
    half = True, # Enable mixed precision for speed-up
    name = "final_model_with_best_params" # Unique experiment name for logging
)

# Evaluate the trained model on the validation set
metrics = model.val(data = os.path.join(ROOT_DIR, "config.yaml"))
print("Validation metrics:")
print(metrics.results_dict)

engine/trainer: task=detect, mode=train, model=/content/drive/My Drive/YOLO_Training/yolo11m.pt, data=/content/drive/My Drive/YOLO_Training/config.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=800, save=True, save_period=-1, cache=False, device=cuda:0, workers=2, project=None, name=final_model_with_best_params, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

100%|██████████| 755k/755k [00:00<00:00, 14.3MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 64.7MB/s]


AMP: checks passed ✅


...


100 epochs completed in 2.682 hours.
Optimizer stripped from runs/detect/final_model_with_best_params/weights/last.pt, 40.6MB
Optimizer stripped from runs/detect/final_model_with_best_params/weights/best.pt, 40.6MB

Validating runs/detect/final_model_with_best_params/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.37it/s]


                   all         45       1281      0.973       0.99      0.989      0.792
                X Axis         30         30      0.983          1      0.995      0.941
                Y Axis         30         30      0.983          1      0.995      0.889
                 Title         45         45       0.99          1      0.995        0.7
                Legend         45         45      0.989          1      0.995      0.925
             Bar Chart         15         15      0.965          1      0.995      0.995
            Line Chart         15         15      0.965          1      0.995      0.995
             Pie Chart         15         15      0.967          1      0.995      0.912
           Data Points         45        640      0.993      0.963      0.993      0.582
          X Axis Ticks         29        213      0.959          1      0.962      0.614
          Y Axis Ticks         29        173      0.948      0.952      0.968      0.521
          X Axis Titl

val: Scanning /content/drive/My Drive/YOLO_Training/capstone_project/labels/val.cache... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:04<00:00,  1.35it/s]


                   all         45       1281      0.973      0.991      0.989       0.79
                X Axis         30         30      0.983          1      0.995      0.926
                Y Axis         30         30      0.983          1      0.995      0.893
                 Title         45         45       0.99          1      0.995      0.698
                Legend         45         45      0.989          1      0.995      0.922
             Bar Chart         15         15      0.965          1      0.995      0.995
            Line Chart         15         15      0.965          1      0.995      0.995
             Pie Chart         15         15      0.966          1      0.995      0.912
           Data Points         45        640      0.992      0.963      0.992       0.58
          X Axis Ticks         29        213      0.959          1      0.964      0.615
          Y Axis Ticks         29        173      0.954       0.96      0.973      0.525
          X Axis Titl

In [ ]:
import os
import shutil

source = '/content/runs'
destination = '/content/drive/MyDrive/YOLO_Training/'

# Create the destination folder if it doesn't exist
os.makedirs(destination, exist_ok=True)

# Copy all contents from 'runs' to 'YOLO_Training'
for item in os.listdir(source):
    source_item = os.path.join(source, item)
    destination_item = os.path.join(destination, item)
    if os.path.isdir(source_item):
        shutil.copytree(source_item, destination_item)
    else:
        shutil.copy2(source_item, destination_item)

# Additional 70 epoch training

In [ ]:
# Load the trained model
model_path = '/content/drive/MyDrive/YOLO_Training/detect/final_model_with_best_params/weights/best.pt'
model = YOLO(model_path)

In [ ]:
results = model.train(
    data = os.path.join(ROOT_DIR, "config.yaml"), # Dataset config
    epochs = 55, # Additional epochs
    imgsz = 800, # Image size
    batch = 8, # Batch size
    lr0 = best_lr, # Learning rate
    momentum = best_momentum, # Momentum
    weight_decay = best_wd, # Weight decay
    workers = 2, # Number of dataloader workers
    half = True, # Enable mixed precision for speed-up
    optimizer = "SGD", # Optimizer
    name = "final_model_continued",  # Custom name for this run
    save_dir = "/content/drive/MyDrive/YOLO_Training/runs"  # Save the entire run to your Google Drive
)

# Evaluate the trained model on the validation set
metrics = model.val(data = os.path.join(ROOT_DIR, "config.yaml"))
print("Validation metrics:")
print(metrics.results_dict)

Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/YOLO_Training/detect/final_model_with_best_params/weights/best.pt, data=/content/drive/My Drive/YOLO_Training/config.yaml, epochs=55, time=None, patience=100, batch=8, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=final_model_continued, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,

train: Scanning /content/drive/My Drive/YOLO_Training/capstone_project/labels/train.cache... 1489 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1489/1489 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/My Drive/YOLO_Training/capstone_project/labels/val.cache... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<?, ?it/s]


Plotting labels to runs/detect/final_model_continued/labels.jpg... 
optimizer: SGD(lr=0.0005, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 800 train, 800 val
Using 2 dataloader workers
Logging results to runs/detect/final_model_continued
Starting training for 55 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


55 epochs completed in 1.522 hours.
Optimizer stripped from runs/detect/final_model_continued/weights/last.pt, 40.6MB
Optimizer stripped from runs/detect/final_model_continued/weights/best.pt, 40.6MB

Validating runs/detect/final_model_continued/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.46it/s]


                   all         45       1281       0.97      0.991      0.989      0.792
                X Axis         30         30      0.981          1      0.995      0.931
                Y Axis         30         30      0.982          1      0.995      0.893
                 Title         45         45      0.968      0.978      0.993      0.729
                Legend         45         45      0.988          1      0.995      0.926
             Bar Chart         15         15      0.962          1      0.995      0.995
            Line Chart         15         15      0.962          1      0.995      0.995
             Pie Chart         15         15      0.964          1      0.995      0.924
           Data Points         45        640      0.985      0.975      0.993      0.579
          X Axis Ticks         29        213      0.958          1      0.966      0.606
          Y Axis Ticks         29        173      0.952      0.971      0.975      0.527
          X Axis Titl

val: Scanning /content/drive/My Drive/YOLO_Training/capstone_project/labels/val.cache... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:05<00:00,  1.10it/s]


                   all         45       1281      0.971      0.988      0.989      0.792
                X Axis         30         30      0.983          1      0.995      0.927
                Y Axis         30         30      0.983          1      0.995      0.896
                 Title         45         45      0.969      0.978      0.994      0.721
                Legend         45         45      0.988          1      0.995      0.924
             Bar Chart         15         15      0.963          1      0.995      0.995
            Line Chart         15         15      0.964          1      0.995      0.995
             Pie Chart         15         15      0.965          1      0.995      0.924
           Data Points         45        640      0.987      0.969      0.992      0.578
          X Axis Ticks         29        213      0.958          1      0.966      0.614
          Y Axis Ticks         29        173      0.953      0.947      0.974      0.529
          X Axis Titl

In [ ]:
import os
import shutil

source = '/content/runs'
destination = '/content/drive/MyDrive/YOLO_Training/run2/'

# Create the destination folder if it doesn't exist
os.makedirs(destination, exist_ok=True)

# Copy all contents from 'runs' to 'YOLO_Training'
for item in os.listdir(source):
    source_item = os.path.join(source, item)
    destination_item = os.path.join(destination, item)
    if os.path.isdir(source_item):
        shutil.copytree(source_item, destination_item)
    else:
        shutil.copy2(source_item, destination_item)